In [4]:
# 1. Imports
from src.load_data import load_interactions, filter_min_activity 
from src.split import temporal_train_val_split, get_val_ground_truth 
from src.metrics import recall_at_k, precision_at_k 
from src.baselines import PopularityRecommender, ItemKNNRecommender
from src.config import DATA_DIR

In [5]:
# 2. Load + filter 
df = load_interactions(DATA_DIR, nrows = 100_000) 
df = filter_min_activity(df, min_user_interactions=3, min_item_interactions=3)

In [6]:
# 3. Train/validation split train_df, 
train_df, val_df = temporal_train_val_split(df, val_fraction=0.2) 
val_truth = get_val_ground_truth(val_df)

In [7]:
# 4. Fit baselines 
pop = PopularityRecommender(top_k=1000).fit(train_df) 
knn = ItemKNNRecommender(n_neighbors=50).fit(train_df)

In [8]:
# 5. Generate recommendations for test users 
val_users = list(val_truth.keys())
pop_preds = [pop.recommend(u, k=10) for u in val_users] 
knn_preds = [knn.recommend(u, k=10) for u in val_users] 
y_true = [val_truth[u] for u in val_users]

In [9]:
# 6. Evaluate
for name, preds in [("Popularity", pop_preds), ("ItemKNN", knn_preds)]:
    r10 = recall_at_k(y_true, preds, k=10)
    p10 = precision_at_k(y_true, preds, k=10)
    print(f"{name}  | Recall@10: {r10:.3f}  Precision@10: {p10:.3f}")

Popularity  | Recall@10: 0.214  Precision@10: 0.025
ItemKNN  | Recall@10: 0.133  Precision@10: 0.016
